In [1]:
"""
Script compares Companies from ConnectWise and ZoomInfo to show unique ones
Creation date: 20-03-2024
Last update: 04-07-2024
Created by: Kamil Smolag
To get CW file - CW - System - Report Writer - Customers (Tab) - "All contacts with Website"
"""

import pandas as pd
import datetime
import os

def zoominfo_intent_data_to_df(file_path):
    if os.path.splitext(file_path)[1] == '.csv':
        df_zoominfo = pd.read_csv(file_path)
    elif os.path.splitext(file_path)[1] == '.xlsx':
        df_zoominfo = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")
    df_zoominfo['Source'] = 'ZoomInfo'
    return df_zoominfo

def zoominfo_not_intent_data_to_df(file_path):
    if os.path.splitext(file_path)[1] == '.csv':
        df_zoominfo = pd.read_csv(file_path)
    elif os.path.splitext(file_path)[1] == '.xlsx':
        df_zoominfo = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")
    df_zoominfo['Source'] = 'ZoomInfo'
    return df_zoominfo

def cw_data_to_df():
    df_cw = pd.read_csv("All Contacts with website in eMS.csv")
    df_cw['Source'] = 'ConnectWise'
    return df_cw
    
def append_zoominfo_to_cw(zi_file_type, zi_file_path, df_cw=cw_data_to_df(), ):
    columns_map_not_intent = {
        'Source':'Source',
        'Company Name':'Company Name', 
        'Website':'Website URL', 
        'First Name':'First Name', 
        'Last Name':'Last Name',
        'Email Address':'Default Email',  
        'Direct Phone Number':'Default Phone', 
        }
    # columns_map_intent = {
    #     'Source':'Source',
    #     'Company Name':'Company Name', 
    #     'Company-\nWebsite URL':'Website URL', 
    #     'Contact-\nFirst Name':'First Name', 
    #     'Contact-\nLast Name':'Last Name',
    #     'Contact - Email address':'Default Email',  
    #     'Contact-\nDirect Phone':'Default Phone', 
    #     }
    columns_map_intent = {
        'Source':'Source',
        'Company Name':'Company Name', 
        'Website':'Website URL', 
        'Recommended Contact First Name':'First Name', 
        'Recommended Contact Last Name':'Last Name',
        'Recommended Contact Email':'Default Email',  
        'Recommended Contact Phone (Direct)':'Default Phone', 
        }
    if zi_file_type == "intent":
        df_zoominfo = zoominfo_intent_data_to_df(zi_file_path)
        columns_mapping = columns_map_intent
    elif zi_file_type == "not_intent":
        df_zoominfo = zoominfo_not_intent_data_to_df(zi_file_path)
        columns_mapping = columns_map_not_intent
    else:
        print("ZoomInfo file type is wrong - 'intent' or 'not_intent'")
    
    # columns_to_keep = [col for col in df_zoominfo.columns if col in columns_mapping]
    # df_zoominfo = df_zoominfo[columns_to_keep]  # Keeping only necessery columns
    df_zoominfo.rename(columns=columns_mapping, inplace=True)  # Renaming columns
    df_appended = df_cw.append(df_zoominfo)
    # df_appended.to_excel(f"test_1.xlsx")
    return df_appended

def map_to_cw_template(df):
    columns_map_to_cw_template = {
        'Company Name':'CompanyName',
        '':'CompanyID', 
        '':'Company-\nPhoneNumber', 
        '':'Company-\nFaxNumber',
        '':'Company-\nSiteName',
        '':'Company-\nAddress-Line1',
        '':'Company-\nAddress-Line2',
        '':'Company-\nCity',
        '':'Company-\nState',
        '':'Company-\nZip',
        '':'Company-\nCountry',
        '':'Company-\nWebsiteURL',
        '':'Company-\nAnnualRevenue',
        '':'Company-\nNumberofemployees',
        '':'Company-\nSICcode',
        '':'Company-\nSICcode',
        '':'Company-TaxID',
        '':'Company-TaxCode',
        '':'Company-Account',
        '':'Company-VendorID',
        '':'Company-Note',
        '':'Contact-\nFirstName',
        '':'Contact-\nLastName',
        '':'Contact-JobTitle',
        '':'Contact-\nDirectPhone',
        '':'Contact-\nDirectExtension',
        '':'Contact-Emailaddress',
        '':'Contact-\nHomephone',
        '':'Contact-\nCellPhone',
        '':'Contact-\nNote',
        '':'Contact-\nActivityAssignToMember',
        '':'Contact-\nActivitySubject',
        '':'PrimaryContact',
        '':'BillingContact',
        }

def define_duplicates(zi_file_type, zi_file_path):
    df_appended = append_zoominfo_to_cw(zi_file_type, zi_file_path)
    now = datetime.datetime.now()  # Get the current date and time
    date_time_str = now.strftime("%Y-%m-%d_%H-%M")
    date_str = now.strftime("%Y-%m-%d")
    # Identify duplicate rows based on the 'Default Email' column
    duplicates_mask = df_appended.duplicated(subset=['Default Email'], keep=False)
    df_unique = df_appended[~duplicates_mask]  # DataFrame with all unique values
    df_unique = df_unique[df_unique['Source'] != 'ConnectWise']  # Drop rows where 'Source' is 'ConnectWise' from df_unique
    df_duplicates = df_appended[duplicates_mask]  # DataFrame with all duplicate values
    df_duplicates = df_duplicates[df_duplicates['Source'] != 'ConnectWise']  # Drop rows where 'Source' is 'ConnectWise' from df_unique
    if not os.path.exists(date_str):  # Check if the directory already exists
        os.makedirs(date_str)  # Create a directory if it does not exist
        
    zi_file_name = os.path.basename(zi_file_path).split('_')[1]
    df_duplicates.drop('Source', axis=1).to_excel(f"{date_str}/duplicate_contacts_{zi_file_name}_{date_time_str}.xlsx", index=False)
    df_unique.drop('Source', axis=1).to_excel(f"{date_str}/unique_contacts_{zi_file_name}_{date_time_str}.xlsx", index=False)
    print("Files saved!")
    return df_unique.drop('Source', axis=1)

"""
Choose zi_file_type = "intent" or "not_intent"
"""

zi_file_type = "intent"
zi_file_folder = r"C:\eMazzanti\zoominfo to cw\ZI_file\\"
for filename in os.listdir(zi_file_folder):
    if filename.startswith('ZI_'):
        zi_file_path = filename
        define_duplicates(zi_file_type, zi_file_folder + zi_file_path)
        # df_unique = define_duplicates(zi_file_type, zi_file_folder + zi_file_path)


Files saved!
